# Prepare Divvy Spatial Data
2/17/24
This notebook prepares Divvy ride data for spatial analysis.

### Overview of Data
868,160 rides have missing data for start station, but these all have GPS (lat/lng) coordinates and are likely undocked electric bikes. So I decided to group bikes by start_station_name to get station-level summary data, but create a separate file for all rides starting with free-floating bikes (~15% of bikes) not docked in a station.

For unknown reasons, the GPS location of undocked free-floating bikes is provided with low precision, to only two decimal places (e.g. 41.91,-87.63), which limits the number of possible locations to 730 citywide even though actual bike locations are limitless. So these rides can be rolled up to "pseudo-stations."

This creates three files in the folder /data/02-prepped/:
| dataset | description | # records | record type |
|---|:---|---|:---|
|divvy-rides-2023-02-to-2024-01.csv|all rides|5,674,449|rides|
|divvy-non-station-rides-2023-02-to-2024-01.csv|rides initiated from free-floating (non-docked) bikes|868,160|rides|
|divvy-stations-2023-02-to-2024-01.csv|rides grouped by starting station, including 4,806,289 rides|1,596|stations|
|divvy-undocked_locations-2023-02-to-2024-01.csv|undocked rides grouped by GPS coordinates, including 868,160 rides|45|locations|

and another file for monthly totals in /results/rides-by-month.csv

# Read Full Year

In [1]:
import pandas as pd

In [29]:
df = pd.read_csv("../data/01-raw/full-year.csv")

In [38]:
df.head()

ride_id  rideable_type           started_at             ended_at  \
0  CBCD0D7777F0E45F   classic_bike  2023-02-14 11:59:42  2023-02-14 12:13:38   
1  F3EC5FCE5FF39DE9  electric_bike  2023-02-15 13:53:48  2023-02-15 13:59:08   
2  E54C1F27FA9354FF   classic_bike  2023-02-19 11:10:57  2023-02-19 11:35:01   
3  3D561E04F739CC45  electric_bike  2023-02-26 16:12:05  2023-02-26 16:39:55   
4  0CB4B4D53B2DBE05  electric_bike  2023-02-20 11:55:23  2023-02-20 12:05:48   

             start_station_name start_station_id  \
0  Southport Ave & Clybourn Ave     TA1309000030   
1    Clarendon Ave & Gordon Ter            13379   
2  Southport Ave & Clybourn Ave     TA1309000030   
3  Southport Ave & Clybourn Ave     TA1309000030   
4   Prairie Ave & Garfield Blvd     TA1307000160   

                end_station_name end_station_id  start_lat  start_lng  \
0         Clark St & Schiller St   TA1309000024  41.920771 -87.663712   
1     Sheridan Rd & Lawrence Ave   TA1309000041  41.957879 -87.649584   
2        Aberdeen St & Monroe St          13156  41.920771 -87.663712   
3  Franklin St & Adams St (Temp)   TA1309000008  41.920873 -87.663733   
4    Cottage Grove Ave & 63rd St   KA1503000054  41.794827 -87.618795   

     end_lat    end_lng member_casual  
0  41.907993 -87.631501        casual  
1  41.969517 -87.654691        casual  
2  41.880419 -87.655519        member  
3  41.879434 -87.635504        member  
4  41.780531 -87.605970        member

In [39]:
len(df)

5674449

# Prep Data

### check for null stations
My takeaway- a substantial # of divvy rides don't have a station ID, possibly because they're electric bikes tied up in areas outside of stations
notably, all of these have GPS coordinates

In [40]:
# check for null stations
df['start_station_name'].isnull().sum()

868160

In [41]:
# check for null stations
df['start_lat'].isnull().sum()

0

In [42]:
# check for null stations
df['start_lng'].isnull().sum()

0

In [43]:
# check for null stations
df['start_station_id'].isnull().sum()

868292

In [44]:
# check for null stations
df['end_station_name'].isnull().sum()

922111

In [45]:
# check for null stations
df['end_station_id'].isnull().sum()

922252

In [46]:
# check for null stations
df['end_lat'].isnull().sum()

7151

In [47]:
# check for null stations
df['end_lng'].isnull().sum()

7151

### label undocked stations

In [48]:
# file undocked
df['start_station_name'] = df['start_station_name'].fillna('undocked')
df['end_station_name'] = df['start_station_name'].fillna('undocked')

### parse dates

In [49]:
# convert times to datetime
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [51]:
# parse date and time
df['start_date'] = df['started_at'].dt.strftime('%Y-%m-%d')
df['start_time'] = df['started_at'].dt.strftime('%H:%M')
df['year_month'] = df['started_at'].dt.strftime('%Y-%m')

### flag for January and January 14-16

In [52]:
df['is_january']=(df['start_date']>='2024-01-01') & (df['start_date']<='2024-01-31')

In [53]:
df['is_jan14_16']=(df['start_date']>='2024-01-14') & (df['start_date']<='2024-01-16')

In [54]:
df.head()

ride_id  rideable_type          started_at            ended_at  \
0  CBCD0D7777F0E45F   classic_bike 2023-02-14 11:59:42 2023-02-14 12:13:38   
1  F3EC5FCE5FF39DE9  electric_bike 2023-02-15 13:53:48 2023-02-15 13:59:08   
2  E54C1F27FA9354FF   classic_bike 2023-02-19 11:10:57 2023-02-19 11:35:01   
3  3D561E04F739CC45  electric_bike 2023-02-26 16:12:05 2023-02-26 16:39:55   
4  0CB4B4D53B2DBE05  electric_bike 2023-02-20 11:55:23 2023-02-20 12:05:48   

             start_station_name start_station_id  \
0  Southport Ave & Clybourn Ave     TA1309000030   
1    Clarendon Ave & Gordon Ter            13379   
2  Southport Ave & Clybourn Ave     TA1309000030   
3  Southport Ave & Clybourn Ave     TA1309000030   
4   Prairie Ave & Garfield Blvd     TA1307000160   

               end_station_name end_station_id  start_lat  start_lng  \
0  Southport Ave & Clybourn Ave   TA1309000024  41.920771 -87.663712   
1    Clarendon Ave & Gordon Ter   TA1309000041  41.957879 -87.649584   
2  Southport Ave & Clybourn Ave          13156  41.920771 -87.663712   
3  Southport Ave & Clybourn Ave   TA1309000008  41.920873 -87.663733   
4   Prairie Ave & Garfield Blvd   KA1503000054  41.794827 -87.618795   

     end_lat    end_lng member_casual  start_date start_time year_month  \
0  41.907993 -87.631501        casual  2023-02-14      11:59    2023-02   
1  41.969517 -87.654691        casual  2023-02-15      13:53    2023-02   
2  41.880419 -87.655519        member  2023-02-19      11:10    2023-02   
3  41.879434 -87.635504        member  2023-02-26      16:12    2023-02   
4  41.780531 -87.605970        member  2023-02-20      11:55    2023-02   

   is_january  is_jan14_16  
0       False        False  
1       False        False  
2       False        False  
3       False        False  
4       False        False

# Roll Up By Stations

In [55]:
#note, I take the minimum lat/lng for each station name, due to slight variations even within the same station
df_stations = df.groupby('start_station_name').agg(
    rides_all=('ride_id', 'count'),
    rides_january=('is_january', 'sum'),
    rides_jan14_16=('is_jan14_16','sum'),
    first_ride=('start_date','min'),
    last_ride=('start_date','max'),
    first_lat=('start_lat','min'),
    first_lng=('start_lng','min')
).reset_index()

In [56]:
df_stations.head()

start_station_name  rides_all  rides_january  rides_jan14_16  \
0         2112 W Peterson Ave        723             11               0   
1                         410          7              0               0   
2               63rd St Beach        985              6               0   
3           900 W Harrison St      12646            449               4   
4  Aberdeen St & Jackson Blvd      14926            544              14   

   first_ride   last_ride  first_lat  first_lng  
0  2023-02-02  2024-01-28  41.991028 -87.683748  
1  2023-04-27  2023-04-27  41.900000 -87.690000  
2  2023-02-12  2024-01-29  41.780818 -87.576407  
3  2023-02-01  2024-01-31  41.842626 -87.704151  
4  2023-02-01  2024-01-31  41.869211 -87.713046

In [57]:
# confirm total number of rides still matches
df_stations['rides_all'].sum()

5674449

In [58]:
# look at count of undocked rides
df_stations[df_stations['start_station_name']=='undocked']

start_station_name  rides_all  rides_january  rides_jan14_16  first_ride  \
1597           undocked     868160          19165              31  2023-02-01   

       last_ride  first_lat  first_lng  
1597  2024-01-31      41.63     -87.94

### create dataframe for docked stations only

In [59]:
df_stations_docked = df_stations[df_stations['start_station_name'] != 'undocked']
df_stations_docked['rides_all'].sum()

4806289

# Roll Up By Undocked Locations
Despite the infinite number of possible GPS positions for undocked bikes, Divvy's dataset only approximates these locations to two decimal places for latitude and longitude. This results in only 730 possible undocked locations.

### create one dataframe for undocked rides only

In [64]:
df_rides_undocked = df[df['start_station_name']=='undocked']
len(df_rides_undocked)

868160

In [68]:
df_locations_undocked = df_rides_undocked.groupby(['start_lat','start_lng']).agg(
    rides_all=('ride_id', 'count'),
    rides_january=('is_january', 'sum'),
    rides_jan14_16=('is_jan14_16','sum'),
    first_ride=('start_date','min'),
    last_ride=('start_date','max')
).reset_index()

In [69]:
df_locations_undocked

start_lat  start_lng  rides_all  rides_january  rides_jan14_16  \
0        41.63     -87.46          1              0               0   
1        41.64     -87.54          2              0               0   
2        41.65     -87.62          1              0               0   
3        41.65     -87.61          2              0               0   
4        41.65     -87.60         17              0               0   
..         ...        ...        ...            ...             ...   
725      42.07     -87.72          7              0               0   
726      42.07     -87.71         16              0               0   
727      42.07     -87.70         33              0               0   
728      42.07     -87.69         79              1               0   
729      42.07     -87.68         64              0               0   

     first_ride   last_ride  
0    2023-08-10  2023-08-10  
1    2023-06-30  2023-10-30  
2    2023-06-18  2023-06-18  
3    2023-04-21  2023-06-03  
4    2023-02-13  2023-08-22  
..          ...         ...  
725  2023-05-18  2023-11-21  
726  2023-04-09  2023-12-24  
727  2023-02-18  2023-12-05  
728  2023-02-06  2024-01-08  
729  2023-03-31  2023-12-03  

[730 rows x 7 columns]

# Summarize by Season

In [87]:
df_monthly = df.groupby('year_month').agg(
    rides_all=('ride_id', 'count'),
).reset_index()
df_monthly

year_month  rides_all
0     2023-02     190445
1     2023-03     258678
2     2023-04     426590
3     2023-05     604827
4     2023-06     719618
5     2023-07     767650
6     2023-08     771693
7     2023-09     666371
8     2023-10     537113
9     2023-11     362518
10    2023-12     224073
11    2024-01     144873

In [88]:
df_monthly['rides_all'].sum()

5674449

# Export for Analysis

In [85]:
# all rides
# this file is mondo, too big to upload to GitHub, and I don't really need it.
#df.to_csv("../data/02-prepped/rides-2023-02-to-2024-01.csv", index=False)

In [81]:
# docked stations
df_stations_docked.to_csv("../data/02-prepped/stations-2023-02-to-2024-01.csv", index=False)

In [70]:
# undocked locations
df_locations_undocked.to_csv("../data/02-prepped/undocked-locations-2023-02-to-2024-01.csv", index=False)

In [86]:
# undocked rides
df_rides_undocked.to_csv("../data/02-prepped/rides-undocked-2023-02-to-2024-01.csv", index=False)

In [89]:
# undocked rides
df_monthly.to_csv("../results/rides-by-month.csv", index=False)